In [1]:
__author__ = 'Raju Kadam'

# Step 0: Initialize the Jira browser environment.
# set variable env_type to specific non-production environment options are: dev | stage | update

# We're using this APIs https://www.selenium.dev/selenium/docs/api/py/api.html
# use atlassian-venv virtual environment. or install selenium library 'pip install selenium'
from selenium import webdriver
from selenium.webdriver import *
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

import click
import random
import os

import time
import datetime
import sys
import traceback

import requests
import shutil

from dotenv import dotenv_values

# Make sure you have config directory with look and feel templates for different environment.
# ❯ ls
# jira_global_custom_color_scheme.default  jira_global_custom_color_scheme.update   wiki_global_custom_colour_scheme.default
# jira_global_custom_color_scheme.stage    logos                                    wiki_global_custom_colour_scheme.dev
#
#
# Also make sure you have environment files available
# ❯ cat .env.stage
# APP_URL=<Base Jira URL>
# REST_API_APP_URL=<This can be same as APP_URL otherwise different REST API END Point needs to be mentioned>
# BEARER_TOKEN=<enter your token aka PAT>
#
env_type = "stage"
config = dotenv_values(f".env.{env_type}")
config_file_location = "./config"

# Disable warnings about not verifying SSL access.
requests.packages.urllib3.disable_warnings()

# Check if the temp file to disable notification schemes for all projects, already exists or not.
disable_project_notifications_status_file = "disabled_notification_schemes_status.txt"
if not os.path.exists(disable_project_notifications_status_file):
    # Create a new file
    with open(disable_project_notifications_status_file, "w") as file:
        file.truncate(0)


class JIRABrowser:
    header_params = {"content-type": "application/json"}
    
    def __init__(self, driver):
        self.browser = driver

    def get_login_elements(self, login_type, base_url):

        return {
                'param_user': 'login-form-username',
                'param_password': 'login-form-password',
                'param_submit': 'login-form-submit',
                'param_login_url': base_url + '/login.jsp?nosso',
                'param_new_base_url': base_url
                }

    # noinspection PyBroadException
    def login(self, login_type, base_url, userid, password):
        browser = self.browser
        login_elem_dict = self.get_login_elements(login_type, base_url)
        #click.echo(login_elem_dict)
        new_base_url = login_elem_dict['param_new_base_url']

        if not self.verify_admin_access():
            try:
                print(f"Login to the application located at {login_elem_dict['param_login_url']}")
                #browser.get(login_elem_dict['param_login_url'])                
                #browser.implicitly_wait(1)
                os_name = browser.find_element(By.ID, login_elem_dict['param_user'])
                os_name.clear()
                os_name.send_keys(userid)

                os_password = browser.find_element(By.ID, login_elem_dict['param_password'])
                os_password.clear()
                os_password.send_keys(password)

                browser.find_element(By.ID, login_elem_dict['param_submit']).click()
                time.sleep(1)

                #click.echo('Login as Admin user')

                # On Premise Atlassian application usually asks Authentication for one more time.
                new_base_url = login_elem_dict['param_new_base_url']
                browser.get(new_base_url + "/secure/admin/ViewApplicationProperties.jspa")
                time.sleep(2)

                # Verify that we are on Administration Console.
                # This will confirm, we are logged in as a Global Administrator.
                assert browser.find_element(By.ID, 'maximumAuthenticationAttemptsAllowed').text.startswith('Maximum Authentication Attempts Allowed')
                
                # TO DO: Need to verify how to relogin if websudo is not disabled.
                #        Also need to find out what we need to do in case of Jira Cloud!
                #if login_type == 'on-premise':
                #    browser.find_element(By.ID, 'login-form-authenticatePassword').send_keys(password)
                #    browser.find_element(By.ID, 'login-form-submit').click()


            except NoSuchElementException:
                click.echo("Unable to login to Jira Application, exiting.")
                traceback.print_exc(file=sys.stdout)
                #browser.close()
                #browser.quit()
                #sys.exit(0)

        return browser, new_base_url

    def verify_admin_access(self):
        browser = self.browser
        try:
            browser.implicitly_wait(1)
            browser.find_element(By.ID, "system-admin-menu")
            return True
        except NoSuchElementException:
            return False

    def get_jira_project_list(self, rest_api_base_url, bearer_token):
        rest_api_header_params = {"content-type": "application/json"}
        rest_api_header_params["Authorization"] = "Bearer " + bearer_token
        jira_project_list_rest_url = rest_api_base_url + "/rest/api/2/project"
        print(f"Get list of all projects: {jira_project_list_rest_url}")
        result = requests.get(jira_project_list_rest_url,  headers=rest_api_header_params, verify=False)
        result.raise_for_status()

        result_len = len(result.json())

        project_id_dict = {}
        for i in range(0, result_len):
            project_id_dict[result.json()[i]['key']] = result.json()[i]['id']
            #click.echo(result.json()[i]['key'] + ":" + result.json()[i]['id'])

        return project_id_dict

    def disable_project_notification_schemes(self, browser, base_url, rest_api_base_url, bearer_token):
        project_notification_url = base_url + '/secure/project/SelectProjectScheme!default.jspa?projectId=%s'
        
        project_dict = self.get_jira_project_list(rest_api_base_url, bearer_token)
        all_project_keys = sorted(project_dict.keys())
        # click.echo(all_project_keys)
        if not all_project_keys:
            click.echo("Unable to reterive project list. May be 'Bearer Token' is not valid")
            return
        
        notification_schemes_disabled_projects = []
        with open(disable_project_notifications_status_file, "r") as notification_scheme_status_temp_file:
            content = notification_scheme_status_temp_file.readline()
            projects = content.split(",")
            # remove any empy strings if we have!
            projects = [ project_key.strip() for project_key in projects if project_key]
            if projects:
                click.echo(f"Looks like we already got list of projects for which notification schemes are disabled: {projects}")
                notification_schemes_disabled_projects = projects
                
        # Let us make sure we only work on projects for which notification scheme is not yet disabled.
        non_disabled_project_set = set(all_project_keys) - set(notification_schemes_disabled_projects)
        project_keys_to_work_on = sorted(list(non_disabled_project_set))
        click.echo(f'This projects we will disable notification schemes: {project_keys_to_work_on}')
        
        try:
            with open(disable_project_notifications_status_file, "w") as write_file:
                for project_key in project_keys_to_work_on:
                    click.echo(f"Disabling notification scheme for project: {project_key}")
                    browser.get(project_notification_url % project_dict[project_key])
                    scheme_dropdown_element = Select(browser.find_element(By.ID, 'schemeIds_select'))
                    current_selected_option = scheme_dropdown_element.first_selected_option
                    current_notification_scheme_name = current_selected_option.text.strip()
                    if current_notification_scheme_name != 'None':
                        scheme_dropdown_element.select_by_visible_text('None')
                        browser.find_element(By.ID, 'associate_submit').click()
                        click.echo("For Project '%s', Notification scheme changed from '%s' to None" % (project_key, current_notification_scheme_name))
                    notification_schemes_disabled_projects.append(project_key)
                    
                click.echo("Done with disabling project notification schemes")
                # reset temp file disable_notification_schemes_status.txt that we used to keep track of projects with disabled notificaition schemes.
                write_file.truncate(0)
        except:
            click.echo("Exception occured! We will save information about projects with disabled notification schemes.")
            with open(disable_project_notifications_status_file, "w") as write_file:
                write_file.write(','.join(notification_schemes_disabled_projects))
            # throw error details to have look at it for debugging.    
            traceback.print_exc() 
        

    def update_global_look_and_feel(self, browser, base_url):
        look_and_feel_url = base_url + '/secure/admin/LookAndFeel!default.jspa'
        browser.get(look_and_feel_url)

        global_custom_color_scheme_dict = {}
        with open(f'{config_file_location}/jira_global_custom_color_scheme.{env_type}') as colour_scheme_file:
            for line in colour_scheme_file:
                lf_element_name, colour_value = line.partition("=")[::2]
                global_custom_color_scheme_dict[lf_element_name] = colour_value.strip()
                
        #Change Jira favicon to reflect the environment color.
        # Changing favicon from user interface is not working. So checking with Atlassian on that.
        # Till then we will comment out this code.
        '''
        favicon_element = browser.find_element(By.ID, "faviconUrl")
        favicon_element.clear()
        favicon.element.send_keys("https://raw.githubusercontent.com/rkadam/automate_atlassian_command_line/python-3.x-compatible-up-to-date-branch/config/logos/s-jira-logo.png")
        favicon_submit_button = browser.find_element(By.ID, "favicon-upload")
        favicon_submit_button.click()
        # sleep for 5 seconds till page get refreshed.
        time.sleep(5)
        '''
        
        jira_lf_elements = sorted(global_custom_color_scheme_dict.keys())
        colour_element = None
        for lf_element_name in jira_lf_elements:
            color_value = global_custom_color_scheme_dict[lf_element_name]
            click.echo ('%s , %s' % (lf_element_name, colour_value))
            # Activate input text field so that we can update the new color for this look and feel element.
            element = browser.find_element(By.XPATH, f'//*[@data-id="{lf_element_name}"]/td[2]')
            element.click()
            # Set new color value for this element.
            text_element = browser.find_element(By.XPATH, f'//*[@data-id="{lf_element_name}"]/td[2]/span/input')
            text_element.clear()
            text_element.send_keys(f'{color_value}')
            # Click on *update* to make sure changes are submitted!
            update_element = browser.find_element(By.XPATH, f'//*[@data-id="{lf_element_name}"]/td[3]/input')
            update_element.click()
            time.sleep(5)
            
        click.echo('Jira Look and Feel updates are completed. Reloading this page to show the new color schemes in effect!')
        browser.get(look_and_feel_url)

    def check_jira_mail_queue_status (self, browser, base_url, mail_threshold_limit):
        click.echo("    Override default mail-threshold-limit (100 emails in queue) if necessary.")
        click.echo("---")
        mail_queue_url = base_url + '/secure/admin/MailQueueAdmin!default.jspa'

        # Visit Mail Queue page
        browser.get(mail_queue_url)
        current_queue_status_text = browser.find_element(By.CLASS_NAME, 'jiraformbody').text
        current_email_in_queue_count = current_queue_status_text.strip().split()[4]
        if int(current_email_in_queue_count) > mail_threshold_limit:
            # TODO: Send Email to Admins
            click.echo('Emails Queued in Jira : %s' % current_email_in_queue_count)
            click.echo('Emails are piling in Jira Mail queue. Please have a look at earliest')
        else:
            click.echo('All is well with Mail Queue!')

    # Be careful with using this method over too many projects or project with too many attachments!
    def get_jira_attachments(self, browser, rest_api_base_url, userid, password, jql, download_dir):
        click.echo("    Override default values to jql (created=now()) and download-dir (./downloads) if necessary.")
        click.echo("---")

        auth = (userid, password)

        #jira_search_rest_url = base_url + "/rest/api/2/search?" + urllib.urlencode(jql) +"&fields=attachment"
        jira_search_rest_url = rest_api_base_url + "/rest/api/2/search?jql=" + requests.utils.quote(jql) +"&fields=attachment"

        #click.echo(jira_search_rest_url)

        issue_starting_index = 0
        total_issue_entries_available = 100
        issue_limit_per_fetch = 50

        while issue_starting_index < total_issue_entries_available:
            updated_jira_search_rest_url = jira_search_rest_url + "&startAt=" + str(issue_starting_index) + "&maxResults=" + str(issue_limit_per_fetch)
            #click.echo(updated_jira_search_rest_url)

            search_result = requests.get(updated_jira_search_rest_url, headers=header_params, auth=auth, verify=False)
            search_result.raise_for_status()

            result_issue_entries = search_result.json()["issues"]
            #click.echo(result_issue_entries)
            result_issue_count_fetch_in_this_iteration = len(result_issue_entries)
            total_issue_entries_available = search_result.json()["total"]

            click.echo("Starting Index - " + str(issue_starting_index) + ", Issues fetched in this iteration - " + str(result_issue_count_fetch_in_this_iteration)
            + ", Total Issues to be fetched - " + str(total_issue_entries_available))
            for i in range(0, result_issue_count_fetch_in_this_iteration):
                # Get Attachment info.
                if 'fields' in result_issue_entries[i] and 'attachment' in result_issue_entries[i]['fields']:
                    attachment_info = result_issue_entries[i]['fields']['attachment']
                    if attachment_info != None:
                        total_attachments = len(attachment_info)
                        for attach_index in range(0, total_attachments):
                            click.echo("Downloading attachment - " + attachment_info[attach_index]['content'] + " for Issue: " + result_issue_entries[i]['key'])

                            attachment_response = requests.get(attachment_info[attach_index]['content'], auth=auth, stream=True)
                            attachment_response.raise_for_status()
                            with open(download_dir + "/" + attachment_info[attach_index]['filename'], 'wb') as f:
                                    attachment_response.raw.decode_content = True
                                    shutil.copyfileobj(attachment_response.raw, f)

            issue_starting_index = search_result.json()['startAt'] + issue_limit_per_fetch

    def check_ldap_sync_status(self, browser, base_url, ldap_sync_threshold_limit):
        click.echo("    Override default ldap-sync-threshold-limit (4) hours if necessary.")
        click.echo("---")

        # If last LDAP sync happened more than ldap_sync_threshold_limit hours ago, warn Jira Admin
        ldap_sync_status_url = base_url + '/plugins/servlet/embedded-crowd/directories/list'
        browser.get(ldap_sync_status_url)

        # Get last successful SYNC time information. Example: Last synchronised at 7/16/15 9:52 AM (took 25s)
        try:
            ldap_sync_info_element = browser.find_element(By.CLASS_NAME, 'sync-info')

            ldap_sync_status_string_aray = browser.find_element(By.CLASS_NAME, 'sync-info').text.strip().split()
            last_successful_sync_status_time = '%s %s %s' % (ldap_sync_status_string_aray[3], ldap_sync_status_string_aray[4], ldap_sync_status_string_aray[5])
            click.echo('Last Successful Sync Status Time: %s' % last_successful_sync_status_time)

            # Do arithmetic to find out how many hours before this sync happened.
            last_sync_datetime = datetime.datetime.strptime(last_successful_sync_status_time, "%m/%d/%y %I:%M %p")
            current_daytime = datetime.datetime.now()
            time_delta = current_daytime - last_sync_datetime
            hours, minutes, seconds = self.convert_timedelta(time_delta)
            sync_status_message = 'Time elapsed since last LDAP sync: {} hour(s), {} minute(s)'.format(hours, minutes)
            click.echo(sync_status_message)

            if hours > ldap_sync_threshold_limit:
                click.echo("Something is wrong with LDAP sync process. Please verify at your earliest your convenience.")

        except NoSuchElementException as e:
            click.echo('Looks like you are not using LDAP or Active Directory! Nothing much to do here...')
        except Exception as e:
            click.echo(e)

    def convert_timedelta(self, duration):
        days, seconds = duration.days, duration.seconds
        hours = days * 24 + seconds // 3600
        minutes = (seconds % 3600) // 60
        seconds = (seconds % 60)
        return hours, minutes, seconds

    command_dictionary = {
        'disable_project_notification_schemes': disable_project_notification_schemes,
        'check_jira_mail_queue_status': check_jira_mail_queue_status,
        'check_ldap_sync_status': check_ldap_sync_status,
        'get_jira_attachments': get_jira_attachments,
        'update_global_look_and_feel': update_global_look_and_feel
    }

@click.command()
# General Parameters needed for Atlassian Command Line use.
@click.option('--app-type', type=click.Choice(['on-premise']),
              default='on-premise', help='->Default: on-premise<-')
@click.option('--app-name', type=click.Choice(['Confluence', 'Jira', 'Bitbucket Server']),
              default='Confluence', help='->Default: Confluence<-')
#"Chrome" is only supported browser as of now. To use ACL in cronjobs, you need to use Chrome with headless settings.
@click.option('--browser-name', type=click.Choice(['Chrome']), default='Chrome', help='Default: ->Chrome<-')
@click.option('--base-url', prompt='Enter Base URL for Atlassian application' )
@click.option('--rest-api-base-url', prompt='Enter REST API Base URL for Atlassian application (usually same as BASE URL)' )
@click.option('--userid', prompt='Enter Administrator Userid')
@click.option('--password', prompt='Enter your credentials', hide_input=True, confirmation_prompt=True)
@click.option('--action', '-a', multiple=True,
              help="Available actions for Confluence ->\n 'update_global_color_scheme', 'update_general_configuration', 'update_wiki_spaces_color_scheme' \n"
                   "---------\n"
                   "Available actions for Jira ->\n 'check_mail_queue_status', 'disable_all_project_notifications', 'check_ldap_sync_status', 'get_jira_attachments'\n -")
# Parameters for Mail Queue Check
@click.option('--mail-threshold-limit', default=100, help="If emails in queue are greater than this limit, then ACL will alert user. ->Default:100<- , Used in Function: check_mail_queue_status()")
# Parameters for LDAP Sync Status check
@click.option('--ldap-sync-threshold-limit', default=4, help="If last LDAP sync happened more than given 'ldap_sync_threshold_limit' hours, then ACL will alert user. ->Default: 4 (hours)<-, Used in Function: check_ldap_sync_status")
# Parameters for Attachment Download
@click.option('--jql', default='created=now()', help='Enter JQL to get attachments for all Jira tickets. ->Default: created = now()<-, Used in Function: get_jira_attachments')
@click.option('--download-dir', default='./downloads', help='Enter complete path for a directory where you want attachments to be downloaded. ->Default Download Directory=./downloads<-, Used in Function: get_jira_attachments')
@click.option('--chrome-driver-location', prompt='Enter complete path for Chrome Driver', help="Make sure you have downloaded Chrome Driver from http://chromedriver.chromium.org/downloads")
@click.option('--wiki-global-color-scheme-file', default='wiki_global_custom_colour_scheme.default', help='Provide name of global color scheme config file for Wiki ->Default config file = wiki_global_custom_colour_scheme.default<-')
def start(app_type, app_name, browser_name, base_url, rest_api_base_url, userid, password, action, mail_threshold_limit, ldap_sync_threshold_limit, jql, download_dir, chrome_driver_location, wiki_global_color_scheme_file):
    """
    'Atlassian Command Line' aka ACL - Automate the tasks which you can not!
    """
    """
    :param string:
    :return:
    """
    click.echo()
    # Remove forward slash from user if user entered in base_url
    base_url = base_url.rstrip('/')
    click.echo('Automating application located at %s' % base_url)
    click.echo()

    #chrome_options = Options()
    #chrome_options.add_argument("--headless")
    #chrome_options.add_argument("--window-size=1366x768")    
    #chrome_driver = chrome_driver_location
    #web_driver = webdriver.Chrome(chrome_options=chrome_options, executable_path=chrome_driver)

    options = Options()
    options.page_load_strategy = 'normal'
    #options.headless = True
    web_driver = webdriver.Chrome(options=options)

    if app_name == 'Confluence':

        wiki_browser = WikiBrowser(web_driver)

        (browser, new_base_url) = wiki_browser.login(app_type, base_url, userid, password)

        for act in action:
            click.echo('Executing Confluence command: %s' % act)
            if act == 'update_global_color_scheme':
                wiki_browser.command_dictionary[act](wiki_browser, browser, new_base_url, "./config/" + wiki_global_color_scheme_file)

            if act == 'update_general_configuration':
                wiki_browser.command_dictionary[act](wiki_browser, browser, new_base_url)

            if act == 'update_wiki_spaces_color_scheme':
                wiki_browser.command_dictionary[act](wiki_browser, browser, new_base_url, userid, password)

            click.echo()

        browser.close()
        browser.quit()

    if app_name == 'Jira':
        jira_browser = JIRABrowser(web_driver)
        (browser, new_base_url) = jira_browser.login(app_type, base_url, userid, password)

        for act in action:
            click.echo('Executing Jira command: %s' % act)
            if act == 'disable_project_notification_schemes':
                jira_browser.command_dictionary[act](jira_browser, browser, new_base_url, rest_api_base_url, userid, password)

            if act == 'check_jira_mail_queue_status':
                jira_browser.command_dictionary[act](jira_browser, browser, new_base_url, mail_threshold_limit)

            if act == 'check_ldap_sync_status':
                jira_browser.command_dictionary[act](jira_browser, browser, new_base_url, ldap_sync_threshold_limit)

            if act == 'get_jira_attachments':
                jira_browser.command_dictionary[act](jira_browser, browser, new_base_url, rest_api_base_url, userid, password, jql, download_dir)
            
            if act == 'update_global_look_and_feel':
                jira_browser.command_dictionary[act](jira_browser, browser, new_base_url)

            click.echo()

        browser.close()
        browser.quit()
        
print(f'Environment set for *{env_type}*. If you want to work in different environment, set variable *env_type* at start of the script.')

Environment set for *stage*. If you want to work in different environment, set variable *env_type* at start of the script.


In [2]:
# Step 1
#
# I'm finding challenges to get to Jira login screen with SSO enabled.
# So easy route is use Jupyter Notebook, initalize the classes
# And create new browser session which will lead us to Jira Login screen.
options = Options()
options.page_load_strategy = 'normal'
#options.add_argument('--headless')
web_driver = webdriver.Chrome(options=options)
# Access following URL in the new Chrome browser that just popped up!
#web_driver.get("<environment>/login.jsp?nosso")

In [3]:
# Step 2
#
# Login to Jira manually so that we can get web_driver instance that we can use in subsequent calls.
# Use "Okta FastPass" to authenticate before we login using internal login URL.
# Once you login manually, execute this cell so that we have jira_browser object setup as needed.

jira_browser = JIRABrowser(web_driver)
(browser, new_base_url) = jira_browser.login('on-premise', config["APP_URL"], config["USERID"],config["PASSWORD"])
print(new_base_url)

https://jira-stage.dropboxer.net


In [4]:
# Step 3 - LDAP Sync Status
# Ask Jira to do operations that you like such as check LDAP sync status, mail_queue status or disable project notification schemes etc.
jira_browser.command_dictionary["check_ldap_sync_status"](jira_browser, web_driver, new_base_url, 4)

    Override default ldap-sync-threshold-limit (4) hours if necessary.
---
Looks like you are not using LDAP or Active Directory! Nothing much to do here...


In [ ]:
# Step 3 - Mail Queue Status check
jira_browser.command_dictionary["check_jira_mail_queue_status"](jira_browser, web_driver, new_base_url, 100)

In [5]:
# Step 3 - Update Global look and Feel for the given test Jira environment.
jira_browser.command_dictionary["update_global_look_and_feel"](jira_browser, browser, new_base_url)

jira.lf.gadget.color1 , #e8dfba

jira.lf.hero.button.base.bg.colour , #e8dfba

jira.lf.hero.button.text.colour , #e8dfba

jira.lf.menu.bgcolour , #e8dfba

jira.lf.menu.textcolour , #e8dfba

jira.lf.text.activelinkcolour , #e8dfba

jira.lf.text.headingcolour , #e8dfba

jira.lf.text.linkcolour , #e8dfba

jira.lf.top.bgcolour , #e8dfba

jira.lf.top.hilightcolour , #e8dfba

jira.lf.top.separator.bgcolor , #e8dfba

jira.lf.top.textcolour , #e8dfba

jira.lf.top.texthilightcolour , #e8dfba

Jira Look and Feel updates are completed. Reloading this page to show the new color schemes in effect!


In [ ]:
# Step 4 - Optional if you like to disable project notification schemes.
# If it shows
jira_browser.command_dictionary["disable_project_notification_schemes"](jira_browser, browser, new_base_url, config["REST_API_APP_URL"], config["BEARER_TOKEN"])